In [ ]:
import proplot as plot

import meds
import numpy as np

In [ ]:
from ngmix.medsreaders import MultiBandNGMixMEDS, NGMixMEDS

mlist = {
    b: NGMixMEDS("/Users/beckermr/Downloads/DES2359-6331_r5227p01_%s_pizza-cutter-slices.fits.fz" % b)
    for b in "riz"
}

In [ ]:
mbm = MultiBandNGMixMEDS(mlist=list(mlist.values()))

In [ ]:
import yaml

with open("metadetect-v3.yaml", "r") as fp:
    config = yaml.safe_load(fp.read())
    
preconfig = config.pop("pizza-cutter-preprocessing")

In [ ]:
from pizza_cutter_metadetect.run_metadetect import _do_metadetect, _load_gaia_stars

gaia_stars = _load_gaia_stars(mbm, preconfig)

In [ ]:
import tqdm

for ind in tqdm.trange(9742,9742+1):
    tqdm.tqdm.write("%d" % ind)
    mbobs = mbm.get_mbobs(ind)
    try:
         _do_metadetect(config, mbobs, gaia_stars, 413, ind, preconfig, [True, True, True])
    except Exception as e:
        fig, axs = plot.subplots(nrows=len(mlist), ncols=4, share=0)
        axs.set_title(ind)

        for i, b in enumerate("riz"):
            obs = mbobs[i][0]

            ax = axs[i, 0]
            ax.imshow(
                np.arcsinh(obs.image * np.sqrt(obs.weight)),
                cmap='rocket'
            )
            ax.grid(False)

            ax = axs[i, 1]
            ax.imshow(
                np.arcsinh(obs.bmask),
                cmap='rocket',
            )
            ax.grid(False)    

            ax = axs[i, 2]
            ax.imshow(
                obs.mfrac,
                cmap='rocket',
                vmin=0,
                vmax=0.25,
            )
            ax.grid(False)    

            ax = axs[i, 3]
            ax.imshow(
                obs.psf.image,
                cmap='rocket',
            )
            ax.grid(False)    
        raise e

In [ ]:
import sys
import logging

logging.basicConfig(fp=sys.stdout)
logging.getLogger("ngmix").setLevel(logging.DEBUG)

In [ ]:
from ngmix.metacal import get_all_metacal

get_all_metacal(mbobs[0][0], psf='fitgauss', rng=np.random.RandomState(seed=10))

fig, axs = plot.subplots()

axs.imshow(mbobs[0][0].image)

In [ ]:
%debug

In [ ]:
fig, axs = plot.subplots()
axs.imshow(np.arcsinh(mbobs[0][0].psf.image))

In [ ]:
mbobs[0][0].jacobian

In [ ]:
np.sum(mlist['z']['dudcol'][:, 0] < -1)

In [ ]:
mbobs = mbm.get_mbobs(9742)

In [ ]:
mbobs[2][0].jacobian

In [ ]:
for col in mlist['z'][9742:9742+1].dtype.names:
    print(col, mlist['z'][9742:9742+1][col])